In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

C:\Users\JOSHWIN ISAC\AppData\Local\Temp\ipykernel_19172\2714595723.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import cv2
import numpy as np
path = "D:\\dataset for lab programs\\TB_Chest_Radiography_Database"
classes = {'Normal': 0, 'Tuberculosis': 1}
def preprocess_images(path, classes):
    x ,y =[],[]
    for key in classes:
        pth = os.path.join(path, key)
        for i in os.listdir(pth):
            img = cv2.imread(os.path.join(pth, i), 0)
            img = cv2.resize(img, (128, 128))
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX)
            x.append(img.flatten())
            y.append(classes[key])
    return np.array(x), np.array(y)

x, y = preprocess_images(path, classes)

In [4]:
class_counts = pd.Series(y).value_counts()
print("Class Distribution:")
print(class_counts)

Class Distribution:
0    3500
1     700
Name: count, dtype: int64


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Data augmentation
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)

# Stratified K-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store model evaluation scores
accuracies = []
precisions = []
recalls = []
f1_scores = []

In [6]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Further split training data into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Oversample the minority class in the training set if needed
oversample = RandomOverSampler()
x_train_resampled, y_train_resampled = oversample.fit_resample(x_train, y_train)


In [7]:
# Class distribution check after sampling
class_counts_sampled = pd.Series(y_train_resampled).value_counts()
print("\nClass Distribution after sampling:")
print(class_counts_sampled)



Class Distribution after sampling:
0    2236
1    2236
Name: count, dtype: int64


In [2]:
#custom CNN
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

shape=(224,224,3)
num_classes=2

model1=Sequential()
model1.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization())
# convolutional layer
model1.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization())

model1.add(Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization())

model1.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(BatchNormalization())
# flatten output of conv
model1.add(Flatten())

# hidden layer
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.5))



